In [1]:
import numpy as np
import cv2 as cv
import glob

In [2]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 準備棋盤格 3D 座標點 (Z=0)
pattern_size = (7, 7) # 7*7的腳點

objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

objpoints = [] # 世界座標
imgpoints = [] # 影像座標

images = glob.glob('images/*.png')  # 放你拍的棋盤格照片
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,7), None)

    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # 顯示檢測結果
        cv.drawChessboardCorners(img, (7,7), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()


In [3]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

print("[info] RMS:", ret)
print("[info] camera matrix:\n", mtx)
print("[info] dist coeffs:", dist.ravel())

np.save('camera_mtx.npy', mtx)
np.save('dist_coeffs.npy', dist)


[info] RMS: 1.3922070580883237
[info] camera matrix:
 [[1.19583528e+03 0.00000000e+00 5.81636554e+02]
 [0.00000000e+00 1.18365598e+03 6.43296649e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[info] dist coeffs: [ 2.53786802e-01 -2.45103051e+00 -2.72751087e-03 -3.91572976e-03
  6.35998129e+00]


In [4]:
img = cv.imread(r'images/IMG_7755.PNG')
h, w = img.shape[:2]

# 這裡要先取得 newcameramtx
newcameramtx, roi = cv.getOptimalNewCameraMatrix(
    mtx, dist, (w, h), 1, (w, h)
)

# 才能用 initUndistortRectifyMap
mapx, mapy = cv.initUndistortRectifyMap(
    mtx, dist, None, newcameramtx, (w, h), 5
)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# 裁切 ROI
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)


True

In [5]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print("total error: {}".format(mean_error/len(objpoints)))

total error: 0.174822052341944
